<a href="https://colab.research.google.com/github/PGLavergne/NYTCrosswordPredicter/blob/main/Project_Part_III.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import matplotlib.pyplot as plt

url = 'https://raw.githubusercontent.com/PGLavergne/NYTCrosswordPredicter/main/nytcrosswords.csv'
dataSet = pd.read_csv(url, encoding='latin-1')
dataSet_partial = pd.read_csv(url, encoding='latin-1', nrows=3000)
dataSet

,Date,Word,Clue
0,10/31/2021,PAT,"Action done while saying ""Good dog"""
1,10/31/2021,RASCALS,Mischief-makers
2,10/31/2021,PEN,It might click for a writer
3,10/31/2021,SEP,Fall mo.
4,10/31/2021,ECO,Kind to Mother Nature
...,...,...,...
781568,11/21/1993,NAT,Actor Pendleton
781569,11/21/1993,SHRED,Bit
781570,11/21/1993,NEA,Teachers' org.
781571,11/21/1993,BEG,Petition


In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer, TextDataset, DataCollatorForLanguageModeling
from torch.utils.data import DataLoader
from tqdm.auto import tqdm
import torch

tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')

#my dataset
dataSet_partial.to_csv('partial_dataset.csv', index=False)
file_path = 'partial_dataset.csv'
dataset = TextDataset(tokenizer=tokenizer, file_path=file_path, block_size=128)

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

train_batch_size = 24
epochs = 3
learning_rate = 5e-5

train_loader = DataLoader(dataset, batch_size=train_batch_size, shuffle=True)

optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.train()

for epoch in range(epochs):
    for batch in tqdm(train_loader, desc=f"Epoch {epoch + 1}/{epochs}"):
        batch = batch.to(device)
        outputs = model(input_ids=batch, labels=batch)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

model.eval()

input_clue = input("Enter the clue: ")
input_size = int(input("Enter the character size of the answer: "))

input_encoded = tokenizer(input_clue, return_tensors='pt', padding=True, truncation=True)

# Generate prediction
generated = model.generate(
    input_ids=input_encoded['input_ids'].to(model.device),
    attention_mask=input_encoded['attention_mask'].to(model.device),
    max_length=input_size,
    num_return_sequences=1
)

decoded_output = tokenizer.decode(generated[0], skip_special_tokens=True)
print(f"Predicted word based on the clue '{input_clue}' and character size {input_size}: {decoded_output}")

/usr/local/lib/python3.10/dist-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


Epoch 1/3:   0%|          | 0/17 [00:00<?, ?it/s]